This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [1]:
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import MathematicalProgram, Solve, ToLatex

# Common Lyapunov Analysis for Linear Systems

In [6]:
def common_lyapunov():
    A = []
    if True:
        # Generate random stable matrices.
        num_states = 4
        num_systems = 2
        for i in range(num_systems):
            d = -np.random.rand(
                num_states,
            )
            v = np.random.randn(num_states, num_states)
            A.append(v.dot(np.diag(d).dot(np.linalg.inv(v))))
    else:
        # Example from lecture notes.
        A.append(np.array(((-1, 0.5), (-3, -1))))
        A.append(np.array(((-1, 0.1), (-10, -1))))
        # Interesting for 2D plotting (a two element parameterization of stable
        # linear systems).  Stable iff ab < 1.
        # a = randn;  ab = 2*rand - 1;  b=ab/a;
        # A{i} = [-1 a; b -1];

    # Create the optimization problem.
    prog = MathematicalProgram()

    # Construct an n-by-n positive semi-definite matrix as the decision
    # variables.
    num_states = A[0].shape[0]
    P = prog.NewSymmetricContinuousVariables(num_states, "P")
    prog.AddPositiveSemidefiniteConstraint(P - 0.01 * np.identity(num_states))

    # Add the common Lyapunov conditions.
    for i in range(len(A)):
        # yapf: disable
        prog.AddPositiveSemidefiniteConstraint(
            -A[i].transpose().dot(P) - P.dot(A[i]) - .01 * np.identity(num_states))
        # yapf: enable

    # Add an objective.
    prog.AddLinearCost(np.trace(P))

    # Run the optimization.
    result = Solve(prog)

    if result.is_success():
        P = result.GetSolution(P)
        eigenvalues = np.linalg.eigvals(P)
        display(Markdown(f"eigenvalues of $P = {ToLatex(eigenvalues)}$\n"))
        for i in range(len(A)):
            display(
                Markdown(
                    f"eigenvalues of $\dot P_{i} = {ToLatex(np.linalg.eigvals(A[i].transpose().dot(P) + P.dot(A[i])))}$\n"
                )
            )
    else:
        print("Could not find a common Lyapunov function.")
        print("This is expected to occur with some probability:  not all")
        print("random sets of stable matrices will have a common Lyapunov")
        print("function.")


common_lyapunov()

eigenvalues of $P = \begin{bmatrix} 1.625 \\ 0.403 \\ 0.036 \\ 0.138 \end{bmatrix}$


eigenvalues of $\dot P_0 = \begin{bmatrix} -2.962 \\ -0.098 \\ -0.010 \\ -0.010 \end{bmatrix}$


eigenvalues of $\dot P_1 = \begin{bmatrix} -1.748 \\ -0.139 \\ -0.010 \\ -0.010 \end{bmatrix}$


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5ec0aeb-d006-4689-a009-180923e76318' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>